In [2]:
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore


In [3]:
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)


In [4]:
# preprocessing train images
train_generator=train_data_gen.flow_from_directory(
     'datasets/train',   # label all images inside angry as angry and with all 
     target_size=(48,48),
     batch_size=64,
     color_mode="grayscale",
     class_mode='categorical'
)


# preprocessing test images
validation_generator=validation_data_gen.flow_from_directory(
     'datasets/test',   
     target_size=(48,48),
     batch_size=64,
     color_mode="grayscale",
     class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
emotion_model=Sequential()
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.25))
emotion_model.add(Dense(7,activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001,decay=1e-6),metrics=['accuracy'])



c:\Users\kushr\OneDrive\Desktop\Emotion Detection Project\emotiondetect\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
